In [21]:
import pandas as pd
import numpy as np

In [22]:
siglas = [['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO'], ['BRASIL']]

In [23]:
colunas = []

colunas_drop_candidato = [
    'DT_GERACAO', 'HH_GERACAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'CD_ELEICAO',
    'DS_ELEICAO', 'DT_ELEICAO', 'TP_ABRANGENCIA_ELEICAO', 'SG_UE', 'NM_UE',
    'CD_CARGO', 'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO',
    'DS_EMAIL', 'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA', 'TP_AGREMIACAO',
    'NM_PARTIDO', 'NR_FEDERACAO', 'NM_FEDERACAO', 'SG_FEDERACAO', 'DS_COMPOSICAO_FEDERACAO',
    'SQ_COLIGACAO', 'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'SG_UF_NASCIMENTO',
    'DT_NASCIMENTO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
    'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL', 'DS_ESTADO_CIVIL',
    'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO', 'DS_OCUPACAO', 'CD_SIT_TOT_TURNO',
    'DS_SIT_TOT_TURNO'
]
colunas.append(colunas_drop_candidato)

colunas_drop_voto = [
    'DT_GERACAO', 'HH_GERACAO', 'CD_TIPO_ELEICAO',
    'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
    'TP_ABRANGENCIA', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
    'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO',
    'NM_VOTAVEL', 'NR_LOCAL_VOTACAO',
    'NM_LOCAL_VOTACAO', 'DS_LOCAL_VOTACAO_ENDERECO', 'NR_VOTAVEL', 'ANO_ELEICAO'
]
colunas.append(colunas_drop_voto)

colunas_drop_extrato = [
    'DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO', 'TP_PRESTACAO_CONTAS', 'DT_PRESTACAO_CONTAS', 'SQ_PRESTADOR_CONTAS', 'SG_UE', 'NM_UE', 'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO', 'NR_CPF_VICE_CANDIDATO', 'CD_TIPO_FORNECEDOR', 'DS_TIPO_FORNECEDOR', 'DS_CNAE_FORNECEDOR', 'NR_CPF_CNPJ_FORNECEDOR', 'NM_FORNECEDOR', 'NM_FORNECEDOR_RFB', 'CD_ESFERA_PART_FORNECEDOR', 'DS_ESFERA_PART_FORNECEDOR', 'SG_UF_FORNECEDOR', 'CD_MUNICIPIO_FORNECEDOR', 'NM_MUNICIPIO_FORNECEDOR','SQ_CANDIDATO_FORNECEDOR', 'NR_CANDIDATO_FORNECEDOR', 'CD_CARGO_FORNECEDOR', 'DS_CARGO_FORNECEDOR', 'DT_DESPESA', 'NR_CPF_CANDIDATO', 'CD_CNAE_FORNECEDOR', 'CD_ORIGEM_DESPESA', 'NR_DOCUMENTO', 'NM_PARTIDO_FORNECEDOR', 'SG_PARTIDO_FORNECEDOR', 'DS_TIPO_DOCUMENTO', 'NR_PARTIDO_FORNECEDOR', 'NR_CANDIDATO', 'NR_PARTIDO', 'NM_PARTIDO', 'SQ_DESPESA', 'DS_DESPESA'
]
colunas.append(colunas_drop_extrato)

In [24]:
nomes = ['consulta_cand_2022_', 'votacao_secao_2022_', 'despesas_contratadas_candidatos_2022_']
upload = ['Candidatos', 'Votos', 'Despesas']
local = ['Candidatos', 'Votos', 'Despesas']
atributos = [['DS_CARGO', 'GOVERNADOR', 'PASSOU_TURNO', 'NR_TURNO'], ['DS_CARGO', 'GOVERNADOR'], ['DS_CARGO', 'Governador']]

In [25]:
def junta(df_list):
    return pd.concat(df_list, ignore_index=True)

In [26]:
def filtra(df, colunas_drop, atributos):
  df = df.drop(columns=colunas_drop)
  df = df.loc[(df[atributos[0]] == atributos[1])]  
  
  if (len(atributos) > 2):
    df[atributos[2]] = np.where(df[atributos[3]] == 1, False, True)   
   
  return df

In [27]:
def tipo(nome, colunas_drop, upload, local, sigla, atributos):
    df_list = []
    
    for s in sigla:
        n = f'{local}\\{nome}{s}.csv'
        df = pd.read_csv(n, delimiter=';', encoding='latin')
        
        df_filtrado = filtra(df, colunas_drop, atributos)
        df_list.append(df_filtrado)
        
        print(n)
        
    df_final = junta(df_list)
    if (upload == 'Candidatos'):
        df_final = df_final.sort_values(by=[atributos[2]], ascending=False).drop_duplicates(subset=['SQ_CANDIDATO'], keep='first')
        df_final = df_final.drop(columns=['NR_TURNO'])
        
    if (upload == 'Despesas'):
        df_final['DS_CARGO'] = df_final['DS_CARGO'].str.upper()
        df_final = df_final.rename(columns={'ST_TURNO': 'NR_TURNO'})
        df_final = df_final[df_final["SG_PARTIDO"] != "PCO"]
        df_final = df_final[df_final["DS_ORIGEM_DESPESA"] != "#NULO"]
    
    df_final.to_csv(upload + '.csv', index=False)

In [28]:
#tipo(nomes[0], colunas[0], upload[0], local[0], siglas[0], atributos[0])
#tipo(nomes[1], colunas[1], upload[1], local[1], siglas[0], atributos[1])
tipo(nomes[2], colunas[2], upload[2], local[2], siglas[1], atributos[2])

Despesas\despesas_contratadas_candidatos_2022_BRASIL.csv
